# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235708428306                   -0.50    7.0         
  2   -7.250224597658       -1.84       -1.39    1.0   9.82ms
  3   -7.251163064661       -3.03       -2.01    2.0   10.9ms
  4   -7.251130038437   +   -4.48       -1.99    2.0   11.9ms
  5   -7.251325957981       -3.71       -2.60    1.0   9.76ms
  6   -7.251337739624       -4.93       -3.06    1.0   10.0ms
  7   -7.251338721053       -6.01       -3.75    1.0   10.2ms
  8   -7.251338793399       -7.14       -4.17    2.0   12.0ms
  9   -7.251338798178       -8.32       -4.68    2.0   11.9ms
 10   -7.251338798638       -9.34       -5.14    2.0   12.3ms
 11   -7.251338798698      -10.22       -5.81    1.0   77.4ms
 12   -7.251338798703      -11.28       -5.98    3.0   13.3ms
 13   -7.251338798704      -12.07       -6.39    1.0   10.4ms
 14   -7.251338798705      -12.74       -6.88    2.0   12.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.045150682862456715
[ Info: Arnoldi iteration step 2: normres = 0.5153799434616646
[ Info: Arnoldi iteration step 3: normres = 0.9840983311836609
[ Info: Arnoldi iteration step 4: normres = 0.30815099027335896
[ Info: Arnoldi iteration step 5: normres = 0.5259385415416025
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.19e-02, 8.25e-02, 4.81e-01, 1.90e-01, 1.03e-02)
[ Info: Arnoldi iteration step 6: normres = 0.25526470747979585
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.11e-02, 1.23e-01, 1.77e-01, 1.15e-01, 2.78e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10489280393087289
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.57e-04, 1.25e-02, 1.47e-02, 5.86e-02, 7.66e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12170015421663354
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.00